### CNN dimensions stuff

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import pandas as pd
import numpy as np
import sys,os,re,csv,pathlib

#import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

%load_ext autoreload
%autoreload 2

In [2]:
c1 = nn.Conv2d(1, 8, kernel_size=(15,2))
m1 = nn.MaxPool2d(kernel_size = (1,2))
c2 = nn.Conv2d(8, 16, kernel_size = (1,2))
m2 = nn.MaxPool2d(kernel_size = (1,2))

cnn = nn.Sequential(c1,m1,c2,m2)

In [3]:
for z in range(12,17):
    print("####\n\nUSING LENGTH = ",z)
    x = torch.rand((1,1,15,z),dtype=torch.float32)
    print("Input: ",x.shape)
    conv1=c1(x)
    print("After Conv1:", conv1.shape)
    pool1=m1(conv1)
    print("After Pool1:", pool1.shape)
    conv2=c2(pool1)
    print("After Conv2:", conv2.shape)
    pool2=m2(conv2)
    print("After Pool2:", pool2.shape)
    if z<15:
        xx = 16*2
    elif z>=15:
        xx = 16*3
    lin = nn.Linear(xx,10)
    
    dense1= lin(pool2.view(-1,xx))
    print("Dense",dense1.shape)
    print("soft",dense1.softmax(1))
    print("arg",dense1.argmax(1))
    print("####\n\n")


####

USING LENGTH =  12
Input:  torch.Size([1, 1, 15, 12])
After Conv1: torch.Size([1, 8, 1, 11])
After Pool1: torch.Size([1, 8, 1, 5])
After Conv2: torch.Size([1, 16, 1, 4])
After Pool2: torch.Size([1, 16, 1, 2])
Dense torch.Size([1, 10])
soft tensor([[0.0929, 0.1150, 0.0869, 0.1024, 0.0870, 0.0853, 0.1370, 0.1060, 0.0725,
         0.1151]], grad_fn=<SoftmaxBackward>)
arg tensor([6])
####


####

USING LENGTH =  13
Input:  torch.Size([1, 1, 15, 13])
After Conv1: torch.Size([1, 8, 1, 12])
After Pool1: torch.Size([1, 8, 1, 6])
After Conv2: torch.Size([1, 16, 1, 5])
After Pool2: torch.Size([1, 16, 1, 2])
Dense torch.Size([1, 10])
soft tensor([[0.0969, 0.1168, 0.0839, 0.0816, 0.1263, 0.1539, 0.1043, 0.1046, 0.0517,
         0.0800]], grad_fn=<SoftmaxBackward>)
arg tensor([5])
####


####

USING LENGTH =  14
Input:  torch.Size([1, 1, 15, 14])
After Conv1: torch.Size([1, 8, 1, 13])
After Pool1: torch.Size([1, 8, 1, 6])
After Conv2: torch.Size([1, 16, 1, 5])
After Pool2: torch.Size([1, 16, 

### Parsing/AA index stuff

In [49]:
AAs=np.array(list('WFGAVILMPYSTNQCKRHDE'))
curPath=os.getcwd()
##AAidx_file='AAindexNormalized.txt' ## AA index reached AUC about 61% for L=14. Worse than AdaBoost
##AAidx_file='AtchleyFactors.txt'  ## Atchley factors work worse than using 544 AA index
AAidx_file='AAidx_PCA.txt' ## works like a charm!!!
gg=open(AAidx_file)
AAidx_Names=gg.readline().strip().split('\t') # Get PC1,... PC15
AAidx_Dict={} # Gets the values for each of the 15 features
i = 0
for ll in gg.readlines():
    if i ==4:
        print(ll)
    
    ll=ll.strip().split('\t')
    AA=ll[0]
    tag=0
    vv=[]
    for xx in ll[1:]:
        vv.append(float(xx))
    if tag==1:
        i+=1
        continue
    i+=1
    AAidx_Dict[AA]=vv
    
Nf=len(AAidx_Dict['C']) #could've just written 15...

pat=re.compile('[\\*_XB]')  ## non-productive CDR3 patterns


C	-8.36918251369427	8.3031934498954	-6.6196694564762	13.8734139891373	8.59531323791867	9.79914818190207	1.26911186662177	-4.63752901352054	-0.983921565990354	4.3634243590938	2.34365256090458	0.343445840449375	1.18290876288827	0.0132558803925318	-0.696621502763574



In [5]:
import pickle
#Save the output of the mapping AAidx_dict in the beginning of DeepCAT.py
with open('AAidx_dict.pkl', 'wb') as f: 
    pickle.dump(AAidx_Dict, f)

In [50]:
np.set_printoptions(precision=3)
AAs=np.array(list('WFGAVILMPYSTNQCKRHDE'))

def OneHotEncoding(Seq):
    Seq_aa = list(Seq) 
    Ns=len(Seq_aa)
    OHE=np.zeros([20,Ns])
    for ii in range(Ns):
        aa=Seq_aa[ii]
        vv=np.where(AAs==aa)
        OHE[vv,ii]=1
    OHE=OHE.astype(np.float32)
    return OHE, vv 
#So in X (row/index) we have the amino acid 
#And in Y (columns) we have the position in the sequence
seq = 'CASSYSTRGGSPLHF'
temp, vv = OneHotEncoding(seq)
print(temp.dtype,"\n", temp.shape,"\n",temp,"\n",type(temp))

float32 
 (20, 15) 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 
 <class 'numpy.nd

In [57]:
from clean_scripts.encoding import *

In [58]:
print(oneHot_encoding('CASSYSGGSPLHF'))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        

In [ ]:
#Nf = number features = 15
Nf=15
def AAindexEncoding(Seq):
    Ns=len(Seq)
    AAE=np.zeros([Ns, Nf])
    for kk in range(Ns):
        ss=Seq[kk]
        #print(ss, kk)
        AAE[kk,]=AAidx_Dict[ss]
    AAE=np.transpose(AAE.astype(np.float32))
    return AAE
seq = 'CASSYSGGSPLHF'
temp = AAindexEncoding(seq)
print(temp,temp.shape)
#print(temp.dtype,"\n", temp.shape,"\n",temp,"\n",type(temp))

In [82]:
llt = np.array([12, 13, 14, 12, 12, 15, 15, 15, 12])

ll = 12
ll1 = 15
[1]*len(np.where(llt==ll)[0])+[0]*len(np.where(llt==ll1)[0])

[1, 1, 1, 1, 0, 0, 0]

In [8]:
def GetFeatureLabels(TumorCDR3s, NonTumorCDR3s):
    nt=len(TumorCDR3s)
    nc=len(NonTumorCDR3s)
    LLt=[len(ss) for ss in TumorCDR3s]
    LLt=np.array(LLt)
    LLc=[len(ss) for ss in NonTumorCDR3s]
    LLc=np.array(LLc)
    NL=range(12,17)
    FeatureDict={}
    LabelDict={}
    for LL in NL:
        vvt=np.where(LLt==LL)[0]
        vvc=np.where(LLc==LL)[0]
        Labels=[1]*len(vvt)+[0]*len(vvc)
        Labels=np.array(Labels)
        Labels=Labels.astype(np.int32)
        data=[]
        for ss in TumorCDR3s[vvt]:
            if len(pat.findall(ss))>0:
                continue
            data.append(AAindexEncoding(ss))
#            data.append(OneHotEncoding(ss))
        for ss in NonTumorCDR3s[vvc]:
            if len(pat.findall(ss))>0:
                continue
            data.append(AAindexEncoding(ss))
#            data.append(OneHotEncoding(ss))
        data=np.array(data)
        features={'x':data,'LL':LL}
        FeatureDict[LL]=features
        LabelDict[LL]=Labels
    return FeatureDict, LabelDict